# Jak měří Semetrika latinský hexametr


## Několik slov k poezii, časomíře a hexametru

Poezii odlišuje od běžné řeči formální pravidelnost:

> ```text
> Seděly žáby v kaluži,
>  - u u  - u    - u u
> hleděly vzhůru k nebi,
>   - u u    - u    - u
> starý jim žabák učený
>   - u  u   - u  - u u
> odvíral tvrdé lebi.
> -  u u    - u  - u
> ```

V tomto úryvku z Nerudových *Písní kosmických* se pravidelně opakují
verše se stejnou posloupností přízvučných (`-`) a nepřízvučných (`u`) slabik
(přízvuk je v češtině obyčejně na první slabice ve slově),
a tedy se stejným rytmem:
první a třetí verš mají posloupnost `-uu-u-uu`,
druhému a čtvrtému proti nim chybí poslední slabika, tedy `-uu-u-u`.
Pravidelné střídání přízvučných a nepřízvučných slabik dává rytmus většině našich básní.

Některým ale místo toho dává rytmus
pravidelné střídání slabik dlouhých a krátkých (označují se stejně, tj. `-` a `u`) neboli *časomíra*:

> ```text
> I draka | měl stráž|cem škare|dým sad | tento ča|rovný.
> -   u u |  -     - | -    u u| -   -  |  -  u  u| -  -
> Byl to pan | inspek|tor. Titu|lem tím | pyšně hla|holným
>  -   u  u  | -   - | -    u u| -   -  |  -  u   u| -  -
> příbuz|ný dale|ký jest | obdaro|ván dvoru | pánem,
>   - - | -  u u| -  -   | -  u u| -    u u |  - -
> myslím | jen, by ti|tul vů|bec něja|ký se mu | dával.
>  -  -  |  -    u  u| -   -| -   u u| -  u  u |  - -
> Střed jeho | inspek|cí pivo|var bý|val s vino|palnou,
>    -   u u | -   - | -  u u| -   -| -     u u| -  -
> kde skoro | z rána po | noc svém | v úřadě | horlivě | trávil.
>   -   u u |    - u  u |  -    -  |   - u u |  -  u u |   - -
> ```

Dlouhá je slabika buďto *přirozeně*,
když je v ní dlouhá samohláska nebo dvojhláska (*ou*),
nebo někdy *pozičně*, když je v ní za samohláskou několik souhlásek;
krátké jsou slabiky ostatní.
Tak např. slovo *la-tin-ský-mi* by se "změřilo" takto `u--u`.

Existuje spousta druhů časoměrných veršů neboli *meter*;
výše uvedený úryvek z básně *Václav Živsa* od Svatopluka Čecha
je složen v *daktylském hexametru*.
Ten se skládá ze šesti *stop*, tj. posloupností slabik dané délky
(jejich hranice se označují `|`).
První čtyři z nich jsou *daktyly* (`-uu`) nebo *spondeje* (`--`);
pátá je skoro vždycky daktyl;
a šestá je spondej nebo daktyl bez poslední slabiky (`-u`).
Daktyl a spondej se mohou zastupovat,
protože v časomíře jedna dlouhá slabika odpovídá délkou dvěma krátkým
(jen v časomíře -- v běžné řeči to ani zdaleka neplatí).
Hexametr tedy můžeme zapsat jako šest čtyřdobých taktů:

![hexametr](hexametr.png)

Kdežto v naší poezii je časomíra vzácná
a většina z nás nedovede vnímat její rytmus
(leda když si pomůžeme nepřirozeným přesouváním přízvuku na první slabiku stopy),
v poezii starého Říma naprosto převažuje.
A její velká část je složena právě v hexametru:
mytologické, naučné, historické i biblické eposy, satiry
i nejdůležitější starořímská báseň, Vergiliova *Aeneida*.

## Shrnutí algoritmu Semetriky

 * vstup: text s každým veršem na zvláštní řádce, nemusí (ale mohou) v něm být označené délky samohlásek, ale musí se v něm rozlišovat *u* a *v* (*i* a *j* nemusí, ale může)
 * výstup: možné hexametrické posloupnosti délek slabik zarovnané s textem verše

 0. vytvoř všechny možné posloupnosti délek slabik v hexametru
 1. znormalizuj vstup
 2. rozděl verš na slova
 3. rozděl slova na samohlásky a souhlásky
    - (a doplň dříve naučené jednoznačné délky samohlásek)
 4. eliduj (tj. když se na hranicích slov setkají dvě samohlásky, první vypusť)
 5. pro každou samohlásku zapiš do rozboru:
    1. `-`, když
       1. je to dvojhláska nebo
       2. je za ní víc než jedna souhláska a není to *muta cum liquida*
    2. `u`, když je krátká a je v otevřené slabice,
    3. jinak `o`
 6. vytvoř všechny možné způsoby, jak v rozboru nahradit značku `o` značkami `-` a `u`
 7. pronikni je s posloupnostmi hexametru (a nečekej, že pokaždé vyjde právě jedna)
 8. zarovnej je s textem verše

In [2]:
from scan import *
from lengths import *
from testing import *

## Vytvoření posloupností hexametru

In [3]:
# schéma hexametru (w: - nebo uu
#                   o  - nebo u)
print(HEXAMETER.scheme)

-w | -w | -w | -w | -uu | -o


 * v prvních čtyřech stopách může být buď daktyl (`-uu`), nebo spondej (`--`)
 * v páté je skoro vždycky daktyl (a Semetrika s ním počítá)
 * v šesté je buďto neúplný daktyl (`-u`), nebo spondej (`--`), ale na délce poslední slabiky nezáleží (říká se, že je *volná*)
 * je tedy celkem 2^4=16 různých posloupností (nebo 2^5=32, když uvážíme i proměnlivost poslední slabiky)

In [4]:
print(len(HEXAMETER.sequences))
# každá druhá posloupnost
# (tj. když pomineme proměnlivost poslední slabiky)
print(sorted(HEXAMETER.sequences.keys())[::2])

32
['---------uu--', '-------uu-uu--', '-----uu---uu--', '-----uu-uu-uu--', '---uu-----uu--', '---uu---uu-uu--', '---uu-uu---uu--', '---uu-uu-uu-uu--', '-uu-------uu--', '-uu-----uu-uu--', '-uu---uu---uu--', '-uu---uu-uu-uu--', '-uu-uu-----uu--', '-uu-uu---uu-uu--', '-uu-uu-uu---uu--', '-uu-uu-uu-uu-uu--']


 * ale pro daný počet slabik jich je nejvýš šest:
   * 13: jediný daktyl je v páté stopě, předtím jsou čtyři spondeje, tedy jediná možnost
   * 14: v prvních čtyřech stopách je jeden daktyl, tedy čtyři možnosti
   * 15: v prvních čtyřech stopách jsou dva daktyly, je (4 nad 2) možností, jak je vybrat, tj. šest
   * dále obdobně
   
 * naproti tomu všech možných posloupností délek slabik je daleko víc:
   * pro 18 slabik: 2^17 = 131 072 (hexametrická je jediná)
   * pro 15 slabik: 2^14 = 16 384 (hexametrických je šest)
   * ale v daném verši je i bez znalosti délek jednoduchých samohlásek počet možných posloupností obyčejně omezen přirozenými délkami dvojhlásek a pozičními délkami

## Normalizace

### Proč?
 - latinské texty se obyčejně píšou latinkou bez diakritiky, a se zpracováním jejich znaků by tedy neměl být problém, ale v učebnicích a slovnících a někdy i jinde se navíc označují ležatou čárkou dlouhé samohlásky (a když je potřeba to zdůraznit, tak krátké obloučkem, např. *fēmĭnă* "žena") a s tím už problém je -- ve znakové sadě Unicode jsou totiž dvě reprezentace písmen s diakritikou: buďto mají jeden znak, nebo dva
 - kromě toho mohou být ve vstupu zbytečné mezery na začátku a na konci a další nepořádné znaky
 
### Jak a co?

 - písmena s diakritikou převádí Semetrika prostřednictvím knihovny `unicodedata` na jeden znak (s tím se pracuje snáz než se dvěma)
   - ale pro y s obloučkem samostatný znak není, a musí se tedy v dalších krocích zvlášť ošetřovat
 - české, šikmé čárky (*fémina*) převádí na ležaté
 - slitky pro dvojhlásky (používané spíš dříve) převádí na dvě písmena
 - maže začáteční a koncové prázdné znaky
 - nechává jen znaky pro zápis latinských slov, interpunkční znaménka a číslice (pro čísla veršů), ostatní znaky maže

In [5]:
femina_po_dvou = "fēmĭnă"
femina_po_jednom = "fēmĭnă"
# z hlediska počítače to jsou dva různé řetězce
print(femina_po_dvou == femina_po_jednom)
# a mají i různou délku
print(len(femina_po_dvou))
print(len(femina_po_jednom))

False
9
6


In [6]:
aeneida1 = Verse(
    "   Arma virumque canó, Tróiæ quí prímus ab órís [1]",
    idle=True)   # aby se verš automaticky nezpracoval
aeneida1.normalize()
aeneida1.normalized_form

'Arma virumque canō, Trōiae quī prīmus ab ōrīs [1]'

## Rozdělení verše na tokeny

 * *token*: slovní tvar nebo interpunkční znaménka (tj. i mezera) nebo číslice (pro čísla veršů)
 
### Proč?

 - interpretace některých písmen závisí na poloze ve slově
   - např. začáteční *i* před samohláskovým písmenem je souhláska [j]
 - některá slova se chovají zvláštně
   - např. ve tvaru *est* následujícím po slově končícím samohláskou se eliduje *e*, kdežto jindy se eliduje vždycky koncová samohláska prvního slova
 - při elidování se pracuje se začáteční a koncovou hláskou slova

### Jak?

 - dvě množiny:
   - písmena pro zápis latinských slov, tj. latinská abeceda
   - ostatní povolená písmena, tj. interpunkční znaménka a číslice
 - Semetrika bere jako slovní tokeny souvislé úseky slovních písmen ohraničené neslovními písmeny; každé neslovní písmeno má pak samostatný token (při měření verše na nich nezáleží)
 
### Problémy
 - kdyby náhodou bylo uprostřed slova interpunkční znaménko (ediční závorky), rozdělí ho na dva tokeny
 - příklonná slova *-que* "a", *-ve* "nebo", *-ne* "zda" se píšou dohromady s předcházejícím slovem, a tímto postupem se tedy neoddělí; ale je dost slov, kde jsou tyto koncové řetězce součástí slov (*quoque* "také")
   - při samém měření to nevadí, ale snižuje to účinnost učení a doplňování délek (k tomu později)

In [7]:
aeneida3 = Verse(
    "litora, multum ille et terris iactatus et alto",
    idle=True)
aeneida3.normalize()
aeneida3.tokenize()
# vytiskne jednotlivé tokeny oddělené svislicemi
aeneida3.print_tokens()

litora|,| |multum| |ille| |et| |terris| |iactatus| |et| |alto


## Rozdělení tokenů na hláskové segmenty

### Proč?

 - co samohláska, to slabika
   - dlouhá jednoduchá samohláska nebo dvojhláska -> přirozeně dlouhá slabika
 - souhlásky
   - víc než dvě souhlásky (až na některé výjimky) -> pozičně dlouhá slabika


### Jak? (a problémy)

 * obecně:
   - tři množiny
     - samohlásková písmena: *a*, *e*, *i*, *o*, *u*, *y* (a diakritika)
     - souhlásková písmena: *b*, *c*, *d*, *f*, *g*, *j*, *k*, *l*, *m*, *n*, *p*, *q*, *r*, *s*, *t*, *v*, *x*, *z*
     - *h*
   - ale zdaleka to není tak jednoduché, že by se daly tokeny rozdělit na písmena a každé jednoznačně zařadit --> Semetrika používá několik pravidel (viz dále) vedoucích k jednoznačnému (ale ne vždycky správnému) rozdělení na segmenty a určení jejich druhu (samohláska/souhláska/*h*)

 * souhláska *h* je zvláštní druh segmentu, protože se nepodílí na poziční délce (např. *adhibeo*) a nebrání elizi
   - (se souhláskami *ph*, *th*, *ch* je zbytečné zacházet zvlášť)

In [8]:
philosophia = Token("philosophia", type_="word")
# oddělí segmenty svislicí a samohlásky napíše velkými písmeny
philosophia.segmentize()
philosophia.print_segments()
(philosophia.segments[0].type_,
 philosophia.segments[1].type_,
 philosophia.segments[2].type_)

p|h|I|l|O|s|O|p|h|I|A


('consonant', 'h', 'vowel')

 * souhlásková písmena
   - *x*, *z* -- dvě souhlásky [ks], [zz] -> dva segmenty
   - *qu* a *gu* ve spojení *ngu*+samohláska -- jedna (labializovaná) souhláska (nezpůsobující sama o sobě poziční délku) -> jeden segment
   - spojení *b*, *p*, *d*, *t*, *c*, *f*, *g* + *r*, *l* (*mūta cum liquidā* "němá souhláska s plynnou") obyčejně nezpůsobuje poziční délku, ale někdy ano; Semetrika je bere jako dva segmenty a řeší je později
   - začáteční skupina dvou a více souhlásek skoro nikdy nezpůsobuje poziční délku poslední slabiky předcházejícího slova; i to Semetrika řeší později

In [9]:
maximus = Token("maximus", type_="word")
maximus.segmentize()
# x jako dva segmenty: "", "x"
maximus.print_segments()
aqua = Token("aqua", type_="word")
aqua.segmentize()
aqua.print_segments()

m|A||x|I|m|U|s
A|qu|A


 * samohlásková písmena
   - *i*
     - začáteční před samohláskou -- souhláska [j]
       - a stejně i po předponě -- je třeba porovnat předcházející úsek se seznamem předpon a doufat, že to je opravdu předpona (skoro vždycky je)
     - mezi dvěma samohláskovými písmeny -- dvě souhlásky [jj]
       - ale ne po *qu*, *ngu* (*quia*, *pinguia*)
     - jinak samohláska [i]
     - problém: básníci občas zaměňují [i] a [j]

In [10]:
maior = Token("maior", type_="word")
maior.segmentize()
maior.print_segments()

m|A||i|O|r


   - nevyřešený problém: *u*
     - trochu podobné jako [i]/[j], ale hůř předvídatelné
     - existuje i minimální pár *uolui* [wolwí] "valil(a) jsem", [woluí] "chtěl(a) jsem" nebo skoro minimální pár *parui* [páruí] "poslechl(a) jsem", [parwí] "malí"
     - básníci zaměňují [u] a [w] stejně jako [i] a [j]
     - ale v novodobých vydáních se často rozlišuje *u* a *v* (i když jednotné psaní *u* se asi šíří), takže na to Semetrika spoléhá
   - *ae*, *oe*, *au* -- většinou dvojhlásky [aj], [oj], [au]
     - problém: ne vždycky -- v řeckých slovech, např. *poeta* [poéta] "básník", *aer* [áér] "vzduch", *Danaum* [dana-um] "(od) Řeků", nebo na morfémovém švu, např. *coeo* [co-eó] "scházím se"
       - občas se v takových případech píše *aë*, *oë* a Semetrika si pak dá poradit
   - *eu*, *ui* jsou v několika málo slovech dvojhlásky
     - problém: *eu* je dvojhláska také pravidelně v řeckých slovech
   - problém: občas básníci stahují dvě samohlásky za sebou do jedné dvojhlásky, např. *deinde* [dejnde] místo [dejinde]
   - koncové spojení samohláska+*m* -- (většinou) dlouhá nosová samohláska
     - nebylo by nutné to zvlášť řešit:
       - buďto je *m* před aspoň jednou souhláskou a pak jakoby způsobují poziční délku
       - anebo je před samohláskou a pak se eliduje (viz dále)
     - ale když se to bere jako jeden segment, tak je pak jednodušší ho elidovat (vyřeší se stejně jako u ústních samohlásek)
     - ale nosové samohlásky uvnitř slova, např. *c**on**sulo* je opravdu zbytečné řešit
     - (i když je to dlouhá samohláska, čárka se nad ni nepíše)
   - největší vadou latinského pravopisu je to, že neoznačuje délku jednoduchých samohlásek -- i když často rozlišuje význam, např. *anus* [anus] "stařena", [ánus] "kruh", slova skloňovaná podle vzoru [fémina] "žena" mají v jednom pádě (nominativ jednotného čísla) koncovku [-a] a v jiném (ablativ) [-á])
     - Semetrika proto později zkouší obě možnosti (ale umí to i chytřeji, viz dále)

In [11]:
for token in aeneida3.tokens:
    token.segmentize()
# vytiskne každý token na zvláštní řádek (_ místo mezery)
aeneida3.print_segments()
# i v iactatus není samohláska
# um v multum jako jeden segment

l|I|t|O|r|A
,
_
m|U|l|t|UM
_
I|l|l|E
_
E|t
_
t|E|r|r|I|s
_
i|A|c|t|A|t|U|s
_
E|t
_
A|l|t|O


## Elize

 * staří Římané neměli rádi, když se na hranicích slov setkaly dvě samohlásky, a proto jednu z nich vypouštěli (elidovali)

### Kdy a jak

   - když se setkají na hranicích slov buďto dvě samohlásky, nebo samohláska a *h* (a za ním v latině nutně následuje samohláska), eliduje se koncová samohláska prvního slova (a začáteční *h* druhého)
     - pravidelná výjimka: neeliduje se citoslovce *o* [ó]
     - za poslední samohláskou prvního slova může být ještě napsáno *m*, ale to ve skutečnosti označuje, že je předcházející samohláska nosová, a Semetrika ho s ní bere jako jeden segment, takže v tomto kroku se tím už nemusí zabývat
   - ale když je druhé slovo *est* nebo *es*, eliduje se místo toho první samohláska v něm
   - Semetrika označí segmenty jako elidované a dá je do závorek

### Problém

 * neeliduje se úplně vždycky

In [12]:
aeneida3.elide()
aeneida3.print_verse()
(aeneida3.tokens[3].segments[-1].lowercase_form,
 aeneida3.tokens[3].segments[-1].elided)

litora, mult(um) ill(e) et terris iactatus et alto


('(um)', True)

## Rozbor slabičných kód

 * slabika: začáteční úsek souhlásek + samohláska + koncový úsek souhlásek (= kóda)

### Proč?

 * pro nalezení pozičních délek -- ty spolu s přirozenými délkami dvojhlásek a dlouhých koncových nosovek zmenšují počet možných meření pro vstup bez označených délek jednoduchých samohlásek

### Jak?

 - spočítat souhláskové segmenty za danou samohláskou (i přes hranice slov) a pamatovat si je
   - nejvýše jeden: *otevřená* slabika (jestliže je v ní jednoduchá ústní samohláska, může být krátká i dlouhá)
     - totéž i tehdy, když jsou všechny souhláskové segmenty součástí až dalšího slova
   - dva, *muta cum liquida*: *obojetná* slabika (totéž)
     - Semetrika si to pamatuje jako jiný případ než předchozí, aby pak mohla správně doplnit délky (viz dále)
   - jinak: *zavřená* slabika, nutně dlouhá (když je samohláska jednoduchá a neznáme její délku, není možné určit, jestli je slabika dlouhá přirozeně, nebo pozičně -- ale pro měření verše na tom nezáleží)
 - u poslední slabiky (ale na její délce v hexametru nezáleží):
   - žádný segment: otevřená
   - jinak: zavřená
 - Semetrika to ukládá k samohláskám

In [13]:
aeneida3.analyse_codas()
(aeneida3.tokens[0].segments[1].coda,   # li[tora] -- otevřená
 aeneida3.tokens[3].segments[1].coda)   # mul[t(um)] -- zavřená

('open', 'closed')

In [14]:
# bez idle=True se verš rovnou zpracuje a změří
aeneida7 = Verse("Albanique patres, atque altae moenia Romae.")
aeneida7.tokens[2].segments[1].coda    # pa[tres] -- obojetná
    # neboli není (zatím) jasné, jestli je otevřená, nebo zavřená

'unknown'

## Vytvoření schématu délek slabik

### Proč?

 * proč rovnou negenerovat možné posloupnosti: aby pak šlo zjistit, na čem Semetrika selhala, když přijde na špatné řešení nebo nepřijde na žádné (Semetrika si také ukládá text se schématem zarovnaným se samohláskami a z něho je pak vidět, které slabiky považuje za jaké a které segmenty považuje za samohlásky)

### Jak?

 * pro každou samohlásku:
   - `-` (dlouhá slabika)
     - když je to dvojhláska nebo nosovka nebo jednoduchá samohláska označená jako dlouhá (přirozená délka)
     - nebo když je v zavřené slabice
   - `u` (krátká slabika)
     - když je to ednoduchá samohláska označená jako krátká a je v otevřené slabice
   - jinak `o` (slabika neznámé délky)
     - tj. jednoduchá samohláska bez označení délky v otevřené nebo obojetné slabice

In [15]:
aeneida7.scheme

'-ooooo----ooo-'

In [16]:
# první prvek v seznamu měření je toto schéma
print(aeneida7.scansions[0][0])
print(aeneida7.scansions[0][1])

Albanique patres, atqu(e) altae moenia Romae.
-  o o  o  o  o   -       -  -   -  oo  o -  


In [17]:
# příklad se všemi délkami označenými
aeneida7_delky = Verse(
    "Albáníque patrés, atque altae moenia Rómae.",
    # považuj neoznačené jednoduché samohlásky za krátké
    unmarked_short=True
)
print(aeneida7_delky.scansions[0][0])
print(aeneida7_delky.scansions[0][1])

Ălbānīquĕ pătrēs, ătqu(ĕ) ăltae moenĭă Rōmae.
-  - -  u  o  -   -       -  -   -  uu  - -  


## Vytvoření možných posloupností délek slabik

 * všechny možné způsoby, jak nahradit `o` za `-` nebo `u`

In [18]:
print(len(aeneida7.candidate_sequences))
print(list(aeneida7.candidate_sequences)[:10])

256
['-uuu-u----u-u-', '--u-uu----u---', '--u--u--------', '----------uuu-', '---u----------', '-u---u----uu--', '--uu----------', '---u-u------u-', '-uuu-------u--', '-uuu------uuu-']


## Nalezení hexametrických posloupností

### Jak?

 * průnik s množinou hexametrických posloupností
 * když se dvě posloupnosti liší jenom délkou poslední, v hexametru volné slabiky, sloučí je Semetrika do jedné (a u poslední slabiky napíše `o`)

In [19]:
# ne pro každý verš se podaří dojít k jedné posloupnosti
aeneida7.metrical_sequences

['---uu-----uu--', '-uu-------uu--']

## Vytisknutí měření

 * Semetrika tiskne pro každé měření text s označenými hranicemi stop a odkrytými délkami (tj. v otevřených slabikách nebo v obojetných změřených jako krátkých) a pod ním měření zarovnané se samohláskami

In [20]:
# správné je první měření
aeneida7.print_scansions()

1. Albā|nīquĕ pă|trēs, at|qu(e) altae | moenĭă | Rōmae.
   -  -| -  u  u|  -   - |      -  -  |  -  uu |  - -  

2. Albănĭ|quē pat\rēs, at|qu(e) altae | moenĭă | Rōmae.
   -  u u|  -  - \ -   - |      -  -  |  -  uu |  - -  


### Jak?

 * kam dát hranice stop (označené svislicí)?
   - teoreticky je to jednoduché: za konec poslední slabiky, tj. u otevřených slabik (kde je za samohláskou nejvýše jedna souhláska) hned za samohlásku, u zavřených (kde jsou aspoň dvě souhlásky) za první souhlásku za samohláskou
   - ale Semetrika nepracuje s fonologickým přepisem, ale obyčejně se zápisem v latinském pravopise bez označených délek a ten přináší tyto problémy:
     + slabika s mutou cum liquida je obyčejně otevřená, leda když je tato skupina na morfémovém švu nebo na hranicích slov
       - ve verši by tedy bylo dobré brát jako zavřené navíc jen ty slabiky, ve kterých je krátká samohláska (tj. které nejsou dlouhé přirozeně, ale pozičně a kde se tedy první souhláska přesouvá do předcházející slabiky)
       - ale když je slabika s jednoduchou samohláskou dlouhá, nepoznáme, jestli proto, že je v ní dlouhá samohláska (tedy slabika je dlouhá přirozeně a je otevřená), nebo že jsou muta cum liquida rozděleny do dvou slabik (a první je tedy dlouhá pozičně a je nutně zavřená)
       - tedy: krátká slabika -> jednoznačně otevřená (a svislice je hned za samohláskou)
       - jinak píše Semetrika zpětné lomítko za první souhlásku, aby naznačila, že si hranicemi stop není jistá
     + u písmen označující dvě souhlásky (*x* [ks], *z* [zz], *i* [jj]) by se přesně měla svislice napsat doprostřed nich, ale protože to nejde, píše se před ně
       - ale když je po *x* ještě další souhláskové písmeno, pak se píše svislice až po *x*
     + když slovo končí spojením krátké samohlásky a souhlásky a další začíná samohláskou, měří se poslední slabika jako krátká, a tedy otevřená a odpovídající dělení by bylo např. "a|b oris", ale když mají hranice stop být v úseku, ve kterém jsou i hranice slov, píše se svislítko do mezery (ale ne v případě elize, která jakoby spojuje dvě slova do jednoho)

In [21]:
# hranice druhé a třetí stopy jsou umístěny, jako by bylo
# ille a et jedno slovo (illet)
# ale hranice páté a šesté slovy jsou v mezeře, i když by
# přesně měly být hned po samohlásce (e|t alto)
aeneida3 = Verse(
    "litora, multum ille et terris iactatus et alto")
aeneida3.print_scansions()

lītŏră, | mult(um) il|l(e) et ter|ris iac|tātŭs ĕt | alto
 - u u  |  -       - |     -   - | -   - | - u  u  | -  o


## Doplnění jednoznačných délek samohlásek

### Proč?

 * když Semetrika pro každou jednoduchou samohlásku v otevřené slabice zkouší obě možné délky, může dojít k víc než jednomu měření (někdy až ke všem možným pro daný počet slabik)
   - anebo může dojít k jednomu, ale špatnému (kvůli špatnému rozdělení na segmenty)
 * kdyby Semetrika aspoň někde délky samohlásek doplnila, pravděpodobnost toho by snížila

### Jak?

 * délka sice často rozlišuje význam (viz výše), ale je také dost případů, kdy jsou délky jednoznačné, např. poslední samohláska ve slově *regina* "královna" může být krátká (v nominativu) i dlouhá (v ablativu), ale první dvě jsou vždycky dlouhé (neexistuje slovo [regina], [regína] ani [régina])

In [22]:
aeneida9 = Verse(
    "quidve dolens, regina deum tot volvere casus")
aeneida9.print_scansions()

1. quidvĕ dŏ|lens, rē|gīnă dĕ|um tot | volvĕrĕ | cāsus
     -  u  u| -     -| - u  u|-   -  |  -  u u |  - - 

2. quidvĕ dŏ|lens, rĕgĭ|nā dē|um tot | volvĕrĕ | cāsus
     -  u  u| -     u u| -  -|-   -  |  -  u u |  - - 


In [23]:
# Semetrika si umí dát poradit
aeneida9_s_delkami = Verse(
    "quidve dolens, régína deum tot volvere casus")
aeneida9_s_delkami.print_scansions()

quidvĕ dŏ|lens, rē|gīnă dĕ|um tot | volvĕrĕ | cāsus
  -  u  u| -     -| - u  u|-   -  |  -  u u |  - - 


### Jak ne?

 - obecné zákonitosti
   - koncové *-i*, *-o*, *-u* je většinou dlouhé (ale ne vždycky)
   - samohláska před koncovým *-t*, *-d*, *-r* je většinou krátká (ale ne vždycky)
   - samohláska před samohláskou je většinou krátká (ale ne vždycky), říká se tomu *vocalis ante vocalem corripitur* ("samohláska před samohláskou se krátí")
   - proč ne: neplatí to obecně a byla by otrava vymýšlet nebo hledat výjimky
 - ručně sestavit slovník jednoznačných délek u nejčastějších slov
   - proč ne: to by taky byla otrava
 - využít nějaký nástroj na doplňování délek
   - proč ne: to bych mohl rovnou využít nějaký nástroj na rozbor hexametru a ušetřit si všechnu tuhle práci
 - nechat Semetriku naučit se délky z velkého korpusu s označenými délkami
   - proč ne: kde ho vzít? a proto, že následující řešení je elegantnější (i když méně účinné)


### Jak ano?

 - nechat Semetriku změřit rozsáhlý korpus hexametrů (bez označených délek)
 - zjistit pro každé slovo v jednoznačně rozebraných verších frekvenci odkrytých délek (jiné se zjistit nedají, ale ani jich není potřeba)
     - tj.: u samohlásek v otevřených slabikách a u krátkých samohlásek před mutou cum liquida
 - a za jednoznačné délky považovat ty, kde má jedna hodnota aspoň nějaký počet a druhá nejvýše nějaký
   - proč připouštět odporující délky: to, že Semetrika dospěje právě k jednomu měření, neznamená, že je správné -- někdy tedy může dojít u nějaké jednoznačné samohlásky ke špatné délce a tím by o ni přišla (ale když připustíme odporující délky, můžeme ovšem naopak nechat Semetriku naučit se nějakou délku jako jednoznačnou, i když skutečně není)
   - jako minimum pro zařazení do slovníku jsem zvolil dvacet a jako maximum odporujících délek tři a obojí jsou divoké odhady
 - tento postup by se mohl i opakovat

 * se slovníkem délek se může počet měření zmenšit nebo zůstat stejný, ale nezvětší se (Semetrika jím nahrazuje jen nejednoznačné délky jednoznačnými)

### Korpus

 * složení:
   * Horatius: *Satiry* a *Listy*
   * Juvenál: *Satiry*
   * Lucan: *Farsalské pole* (historický epos)
   * Lucretius: *O přírodě* (naučný epos)
   * Ovidius: *Proměny* (mytologický epos)
   * Statius: *Achilleida* (mytologický epos)
   * Valerius Flaccus: *Argonautica* (mytologický epos)
   * Vergilius: *Pastýřské zpěvy*, *Rolnické zpěvy* (naučný epos), *Aeneida* (mytologický epos)
 * zdroj: Perseus Digital Library, prostřednictvím CLTK:
   https://github.com/cltk/latin_text_perseus
   (JSON jsem převedl na prostý text)

In [24]:
# počet veršů
!cat perseus_corpus/* | wc -l

56371


In [25]:
default_ld = LengthDictionary()
default_ld.load(".default_length_dictionary.pickle",
               load_frequencies=True)

In [26]:
# počet slov v celém korpusu
len(default_ld.frequencies)

43841

In [27]:
default_ld.frequencies["insequitur"]
# * délka první samohlásky je vždycky neznámá, protože
#   je v zavřené slabice
# * druhá a třetí samohláska je až na jeden případ správně
#   určena jako krátká
# * čtvrtá samohláska je krátká a kdyby následovalo slovo
#   začínající samohláskou, Semetrika by to mohla odhalit,
#   ale v tomto korpusu takový případ není

[{'long': 0, 'short': 0, 'unknown': 36},
 {'long': 1, 'short': 35, 'unknown': 0},
 {'long': 1, 'short': 35, 'unknown': 0},
 {'long': 0, 'short': 0, 'unknown': 36}]

In [28]:
# mimochodem, Semetrika se spletla u tohoto verše
aer = Verse(
    "insequitur candens confestim lucidus aer ,")
aer.print_scansions()

insē|quītur | candens | confes|tim lŭcĭ|dūs aer ,
-  -|  - -  |  -  -   |  -  - | -   u u| -  -    


In [29]:
# s nápovědou, že ve slově aer není dvojhláska,
# už dojde ke správnému měření
aer_s_napovedou = Verse(
    "insequitur candens confestim lucidus aër ,")
aer_s_napovedou.print_scansions()

insĕquĭ|tur can|dens con|festim | lūcĭdŭs | āër ,
-  u  u| -   - | -    - | -  -  |  - u u  | --   


In [30]:
# počet slov vybraných do slovníku délek
len(default_ld.dictionary)

1453

In [31]:
# díky toleranci se dostane toto slovo do slovníku
# se správně označenými délkami
default_ld.dictionary["insequitur"]

['unknown', 'short', 'short', 'unknown']

In [32]:
vzorek = sorted(default_ld.dictionary.keys())[::30]
len(vzorek)

49

In [33]:
for word in vzorek:
    # doplní délky podle slovníku
    default_ld.print_with_lengths(word)

ā
aethĕră
altĕr
ăquārum
bellī
cărīnae
cŏlōres
corpŏrĕ
dăbĭt
dĭānae
dŏmōs
ĕques
făcĭle
fĕrunt
fortĕ
fūnĕră
hăbēnis
hŭmum
impĕrĭum
īram
lăbōre
līmĭtĕ
maiōrĕ
mēcum
mĭnervae
mŏras
nāti
nōmĭnă
occŭpăt
ostĭă
pătrĭo
phoebŭs
pŏsŭit
prō
quārum
rāmis
rīpas
scīrĕ
sĕquī
sōl
stābat
sŭum
tĕnĕt
transīrĕ
ŭmĕros
vĕl
vĕtŭs
vĭrī
vŏlat


 * jestli jsem něco nepřehlédl, jsou v tomto vzorku jen tři nejednoznačné délky určené jako jednoznačné:

   * *Dĭānae* "(bez) Diany", ale také *Dīānae* (hlásková varianta)
   * *nōmĭnă* "jména", ale také *nōmĭnā* "jmenuj"
   * *vĭrī* "mužovi", ale také *vīrī* "(bez) jedu"

 * ale tyto chyby se asi moc často neprojeví

## Malá zkouška úspěšnosti

 * na jedné básni z korpusu na učení délek (první kniha Aeneidy)
 * a na jedné neznámé básni (pátá kniha Avitova biblického eposu)
 * (ne všechny verše s jedním měřením jsou ovšem změřeny správně)

In [34]:
with open("tests/vergil-aeneid1.txt", "r") as file:
    vergilius = file.readlines()

In [35]:
vergilius_test = Test(vergilius)

In [36]:
# sloupce:
# 1) počet měření
# 2) počet veršů tolikrát změřených bez slovníku délek
# 3) ... s ním
# 4) a 5) procenta
vergilius_test.print_statistics()

NUMBER OF VERSES: 756

	| W/O	WITH	| W/O	WITH
0	| 30	39	| 3 %	5 %
1	| 552	653	| 73 %	86 %
2	| 135	55	| 17 %	7 %
3	| 34	8	| 4 %	1 %
4	| 5	1	| 0 %	0 %


In [37]:
with open("tests/avitus.txt", "r") as file:
    avitus = file.readlines()

In [38]:
avitus_test = Test(avitus)

In [39]:
avitus_test.print_statistics()

NUMBER OF VERSES: 719

	| W/O	WITH	| W/O	WITH
0	| 9	12	| 1 %	1 %
1	| 585	642	| 81 %	89 %
2	| 108	61	| 15 %	8 %
3	| 15	3	| 2 %	0 %
4	| 1	0	| 0 %	0 %
6	| 1	1	| 0 %	0 %


Takže nic moc.

## Literatura

 * Allen, Sidney W.: *Vox Latina: A Guide to the Pronunciation of Classical Latin*. London: Cambridge University Press, 1978.
 * Kuťáková, Eva: *Laudabile carmen*. Praha: Karolinum, 2012.
 * Lukeš, David: *How computers handle text: a gentle but thorough introduction to Unicode*. 2016. Online: <https://dlukes.github.io/unicode.html>
 * Panhuis, Dirk: *Gramatika latiny*. Praha: Academia, 2014.

## Jiné nástroje na měření časoměrných veršů

 * http://docs.cltk.org/en/latest/latin.html#scansion-of-poetry
 * https://github.com/blagae/elisio
 * http://www.logical.ai/arma/
 * http://schediazein.altervista.org/Metron/Site/Metron.html
 * http://metronom.jtomaszewski.com/home/
 * http://www.alatius.com/macronizer/
 * https://community.wolfram.com/groups/-/m/t/1732445